In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd 
import re

# Буду смотреть
Проделаем работу, практически полностью аналогичную работе с просмотренными фильмами. На этот раз html-выгрузка содержит больше интересной информации, в т.ч. имя режиссера, жанры и страну выпуска, но с этим разберемся как-нибудь в следующий раз :)

In [67]:
soup = BeautifulSoup(open("D:/MovieDB/CreatingTables/will_watch/0.html", 'r', encoding='utf-8'))

## Запись об одном фильме выглядит как-то так:

    <div class="info">
        <a class="name" href="/film/23712/">Стена</a>
        <span>Pink Floyd: The Wall (1982) 99 мин.</span>
        <span style="margin: 0">
            Великобритания,            <i>реж. <a class="lined" href="/name/28208/">Алан Паркер</a></i>      </span>
        <span>(мюзикл, драма)</span>

        <span><a class="lined" href="/name/77374/">Боб Гелдоф</a>, <a class="lined" href="/name/158491/">Кристин Харгривз</a>, ...</span>
        <span class="last"><a href="/film/23712/sites/">Сайты</a><b> &bull; </b>Афиша<b> &bull; </b><a href="/film/23712/video/">Трейлеры</a><b> &bull; </b><a href="/film/23712/stills/">Кадры</a><b> &bull; </b><a href="">DVD</a>
        </span>
    </div>
    <div class="etc">
        <div class="countries">
                        <div class="viewDate">
                14 июля 1982
                <div class="flag flag8" title="Франция"><span></span></div>
                                    <i class="dvd_2">
                                                                                                </i>
                            </div>
                    </div>
        <div style="padding: 0; margin: 0">
            <p class="rating">
    <div class="vote_widget miniVoteDiv rating_user_23712">&nbsp;</div>
<script nonce="" type="text/javascript">
    ur_data.push({film: 23712, rating: '', user_code: 'b2121f37d09b111ec3a3837ddb8ddc3b'});
        </script>
    <div class="noPosters kpRating  green"><span>8.119 <span>39 732</span></span></div>

In [78]:
regex = re.compile('.*kpRating.*')

infos = soup.findAll('div', {'class': 'info'})
raw_ratings = soup.findAll('div', {'class': regex})

In [122]:
ru_names = []
en_names = []
years = []
ratings = []
ratings_numbers = []
lengths = []

for info, raw_rating in zip(infos, raw_ratings):
    rating_all = raw_rating.findAll('span')
    rating = int(''.join(re.findall('\d+', rating_all[0].text[:5])))
    ratings_number = int(''.join(re.findall('\d+', rating_all[1].text)))
    
    ru_name = info.find('a', {'class': 'name'}).text
    
    other = info.find('span').text
    en_name = ''.join(re.findall('.*\d{4}', other))[:-6]
    year = int(''.join(re.findall('\d{4}', other)))
    length = int((''.join(re.findall('\d+ мин.', other)))[:-5])
    
    ru_names.append(ru_name)
    en_names.append(en_name)
    years.append(year)
    ratings.append(rating)
    ratings_numbers.append(ratings_number)
    lengths.append(length)

In [120]:
def df_templater():
    return {'ru_name': ru_names,
            'en_name': en_names,
            'year': years,
            'rating': ratings,
            'ratings_number': ratings_numbers,
            'length': lengths}    

In [123]:
will_watch = pd.DataFrame(df_templater())
will_watch.head()

,ru_name,en_name,year,rating,ratings_number,length
0,Стена,Pink Floyd: The Wall,1982,8119,39732,99
1,Шоссе в никуда,Lost Highway,1996,7428,48803,134
2,Гордость и предубеждение,Pride & Prejudice,2005,8086,388371,129
3,Отвращение,Repulsion,1965,7621,16771,105
4,Ребенок Розмари,Rosemary's Baby,1968,7663,43521,137


In [125]:
with open('D:\MovieDB\CreatingTables\will_watch\will_watch.csv', 'w', encoding='utf-8') as f:
    f.write(will_watch.to_csv())

При работе с этим файлом у меня появились проблемы с кодировкой, их происхождение пока неясно. Но в любом случае нужно будет учитывать это в дальнейшей работе.